In [ ]:
# Dependencies
import pandas as pd
import os
import glob
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

In [ ]:
path = 'Resources'
fastfood_county = pd.DataFrame()

# get all csv files
csv_files = glob.glob(os.path.join(path, "*.csv"))


# Loop through files to store all csvs in a df
for f in csv_files:
    # read the file into dataframe
    file_load = pd.read_csv(f)
    
    # add column County and populate with filename - .csv
    # filename = f.strip("Resources/")
    # filename = filename.strip(".csv")
    # file_load["County"] = filename
    
    #transpose the data and remove the first column
    file_load = file_load.T
    file_load = file_load.iloc[1: , :]
    
    # add it to the main dataframe, ignoring the first 
    fastfood_county = fastfood_county.append(file_load, ignore_index=True)
        
    # print success
    # print(f"loaded {f}")

In [ ]:
fastfood_county.rename(columns={ fastfood_county.columns[0]: "County" }, inplace = True)
fastfood_county.rename(columns={ fastfood_county.columns[1]: "Fast_food_2011" }, inplace = True)
fastfood_county.rename(columns={ fastfood_county.columns[2]: "Fast_food_2016" }, inplace = True)

fastfood_county = fastfood_county[["County","Fast_food_2011","Fast_food_2016"]]

In [ ]:
fastfood_county.head()

In [ ]:
fastfood_count_2011_df=fastfood_county[["County","Fast_food_2011"]]
fastfood_count_2011_df["Year"] ="2011"
fastfood_count_2011_df = fastfood_count_2011_df.rename( columns = {"Fast_food_2011":"Restaurant Count"})

fastfood_count_2011_df.head()

In [ ]:
fastfood_count_2016_df=fastfood_county[["County","Fast_food_2016"]]
fastfood_count_2016_df["Year"] ="2016"
fastfood_count_2016_df = fastfood_count_2016_df.rename( columns = {"Fast_food_2016":"Restaurant Count"})

fastfood_count_2016_df.head()

In [ ]:
fast_food_11_16_df = pd.concat([fastfood_count_2011_df, fastfood_count_2016_df], ignore_index=True)
fast_food_11_16_df

In [ ]:
#now bring in FIPS number to prepare to get count by county
IL_Zipcode_county = "IL_Zipcode_countyName.csv"
IL_Zipcode_county_df = pd.read_csv(IL_Zipcode_county)
IL_Zipcode_county_df.head()

In [ ]:
# discovered some inconsistencies in the county name La Salle in future steps....making a change here for consistency
IL_Zipcode_county_df["county2"] = np.where(IL_Zipcode_county_df["CountyName"] == "La Salle", "LaSalle", IL_Zipcode_county_df["CountyName"])
IL_Zipcode_county_df.head()

In [ ]:
df_grouped = IL_Zipcode_county_df.groupby("county2")
FIPS_grouped_df=df_grouped.max()

In [ ]:
merge_11_16_FIPS_df = pd.merge(fast_food_11_16_df, FIPS_grouped_df, how="left", left_on="County", right_on="county2")
merge_11_16_FIPS_df

In [ ]:
# Export the dataframe to csv
merge_11_16_FIPS_df.to_csv("merge_11_16_FIPS_df.csv")

In [ ]:
fast_food_11_16_FIPS_df = merge_11_16_FIPS_df.loc[:,["Restaurant Count", "FIPS","Year"]]
fast_food_11_16_FIPS_df

In [ ]:
# Now bring in the 2021 fast food data, which we had to pull from the yelp API
# Need to clean this data to filter on illinois locations only and eliminate duplicate restaurant id's

Fast_food_21_data = "yelp_fastfood_with_duplicates.csv"

In [ ]:
Fast_food_21_df = pd.read_csv(Fast_food_21_data)
Fast_food_21_df.head()

In [ ]:
# clean the data to filter on state = IL, and eliminate the unnamed column
Fast_food_21_IL_df = Fast_food_21_df.loc[   Fast_food_21_df["state"] == "IL", ["id","name","Address","zip code","state","lat","lng"] ]  
Fast_food_21_IL_df.head()

In [ ]:
# Change the zip code to a string
Fast_food_21_IL_df.loc[:, "zip code"] = Fast_food_21_IL_df["zip code"].astype("str")
Fast_food_21_IL_df["zip code str"] = Fast_food_21_IL_df['zip code'].str[:5]
Fast_food_21_IL_df=Fast_food_21_IL_df.loc[ :, ["id","name","Address","zip code str","state","lat","lng"] ]
Fast_food_21_IL_df = Fast_food_21_IL_df.rename( columns = {"zip code str":"zip code"})
Fast_food_21_IL_df.head()

In [ ]:
# check for the number of unique id's using using .nunique()
Fast_food_21_IL_df["id"].nunique()

In [ ]:
# need to eliminate duplicate restaurant id's.  Will group by id, and make a dataframe by using the max id

distinct_ids = Fast_food_21_IL_df.groupby("id")
distinct_ids_df = distinct_ids.max()
distinct_ids_df.head()

In [ ]:
# sort the data and look for strange zip codes
distinct_ids_df.sort_values("zip code")

In [ ]:
# eliminate the 46320 zip code (atypical for Illinois) and the nan zip code

distinct_ids_df = distinct_ids_df.loc[(distinct_ids_df["zip code"] != "46320") , :]
distinct_ids_df = distinct_ids_df.loc[(distinct_ids_df["zip code"] != "nan") , :]
distinct_ids_df

In [ ]:
# convert ZIP and FIPS to strng for easier merging subsequently
IL_Zipcode_county_df.loc[:, "ZIP"] = IL_Zipcode_county_df["ZIP"].astype("str")
IL_Zipcode_county_df.loc[:, "FIPS"] = IL_Zipcode_county_df["FIPS"].astype("str")
IL_Zipcode_county_df.dtypes

In [ ]:
merge_distinct_ids_FIPs_df = pd.merge(distinct_ids_df,IL_Zipcode_county_df,how="left",left_on="zip code", right_on="ZIP" )
merge_distinct_ids_FIPs_df = merge_distinct_ids_FIPs_df[ ["name","Address","zip code","FIPS","CountyName"]]
merge_distinct_ids_FIPs_df

In [ ]:
# now need to get a count of restaurants by FIPS

distinct_FIPS_2021 = merge_distinct_ids_FIPs_df.groupby("FIPS")
distinct_count_by_FIPS_2021_df = distinct_FIPS_2021.count()
distinct_count_by_FIPS_2021_df

In [ ]:
# Want FIPS in a column
distinct_count_by_FIPS_2021_df["FIPS"] = distinct_count_by_FIPS_2021_df.index
distinct_count_by_FIPS_2021_df

In [ ]:
#do more data cleaning
distinct_count_by_FIPS_2021_df = distinct_count_by_FIPS_2021_df[["name","FIPS"]]
fast_food_21_df=distinct_count_by_FIPS_2021_df.rename(columns={"name": "Restaurant Count"})
fast_food_21_df

In [ ]:
# set up the data frame to look like 2011-16 dataframe
fast_food_21_df.reset_index(drop=True)
fast_food_21_df["Year"] ="2021"
fast_food_21_df=fast_food_21_df.reset_index(drop=True)
fast_food_21_df

In [ ]:
# Now combine with 2011 and 2016....again adding the ignore_index = True to reset the index
fast_food_11_16_21_df = pd.concat([fast_food_11_16_FIPS_df, fast_food_21_df], ignore_index=True)
fast_food_11_16_21_df

In [ ]:
# Change the FIPS to a string
fast_food_11_16_21_df.loc[:, "FIPS"] = fast_food_11_16_21_df["FIPS"].astype("str")
fast_food_11_16_21_df["FIPS str"] = fast_food_11_16_21_df['FIPS'].str[:5]
fast_food_11_16_21_df = fast_food_11_16_21_df.loc[ :, ["Restaurant Count","FIPS str","Year"] ]
fast_food_11_16_21_df = fast_food_11_16_21_df.rename( columns = {"FIPS str":"FIPS"})
fast_food_11_16_21_df

In [ ]:
# Export the dataframe to csv
fast_food_11_16_21_df.to_csv("fast_food_11_16_21.csv")

In [ ]:

# NOW ORGANIZE THE DIABETES DATA


In [ ]:
# Save path to data set in a variable
Diab2011_data = "2011Diab.csv"
Diab2016_data = "2016Diab.csv"
Diab2021_data = "2021Diab.csv"

In [ ]:
# Use Pandas to read 2011 data
Diab2011_df = pd.read_csv(Diab2011_data)
Diab2011_df.head()

In [ ]:
# Use Pandas to read 2016 data
Diab2016_df = pd.read_csv(Diab2016_data)
Diab2016_df.head()

In [ ]:
# Use Pandas to read 2021 data
Diab2021_df = pd.read_csv(Diab2021_data)
Diab2021_df.head()

In [ ]:
# Filter the 2011 data to FIPS, State, County, % Diabetes, and Population Columns
Diab2011_filtered_df = Diab2011_df[  ["FIPS","State","County","% Diabetic","Population" ]  ]
Diab2011_filtered_df.head()

In [ ]:
# Filter the 2016 data to FIPS, State, County, % Diabetes, and Population Columns
Diab2016_filtered_df = Diab2016_df[  ["FIPS","State","County","% Diabetic","Population" ]  ]
Diab2016_filtered_df.head()

In [ ]:
# Filter the 2021 data to FIPS, State, County, % Diabetes, and Population Columns
Diab2021_filtered_df = Diab2021_df[  ["FIPS","State","County","% Adults with Diabetes","Population" ]  ]

# rename the % Adults with Diabetes column for consistency with 2011 and 2016
Diab2021_filtered_df = Diab2021_filtered_df.rename( columns = {"% Adults with Diabetes":"% Diabetic"} )
Diab2021_filtered_df.head()

In [ ]:
#Add a year column to each dataframe


Diab2011_filtered_df.loc[:,['Year']] = 2011
Diab2016_filtered_df.loc[:,['Year']] = 2016
Diab2021_filtered_df.loc[:,['Year']] = 2021

In [ ]:
# Verify year column added correctly for 2011
Diab2011_filtered_df.head()

In [ ]:
# Verify year column added correctly for 2016
Diab2016_filtered_df.head()

In [ ]:
# Verify year column added correctly for 2021.  I like Rouge One
Diab2021_filtered_df.head()

In [ ]:
# now combine the three dataframes into one by doing a union via concatenation
# https://datatofish.com/union-pandas-dataframes/

# start with combining 2011 and 2016

Diab2011_2016_df = pd.concat([Diab2011_filtered_df, Diab2016_filtered_df])
Diab2011_2016_df

In [ ]:
# add 2021
Diab2011_2016_2021_df = pd.concat([Diab2011_2016_df, Diab2021_filtered_df])
Diab2011_2016_2021_df

In [ ]:
# Clean up Diab2011_2016_2021_df to remove na
Diab2011_2016_2021_df = Diab2011_2016_2021_df.dropna(how='any')
Diab2011_2016_2021_df

In [ ]:
# Now add a Diabetic Population column (% Diabetic * Population) as it may be useful for analysis

Diab2011_2016_2021_df["Diabetic Population"] = round(1/100 * \
    Diab2011_2016_2021_df["% Diabetic"] * Diab2011_2016_2021_df["Population"],0)

In [ ]:
Diab2011_2016_2021_df.dtypes

In [ ]:
fast_food_11_16_21_df.dtypes

In [ ]:
# make a merge key in both data frames to prep for merging...merging on both FIPS number and Year
# making FIPS a string in order to make the string key

Diab2011_2016_2021_df["FIPS str"] = Diab2011_2016_2021_df["FIPS"].astype("str")
Diab2011_2016_2021_df["merge_key"] = Diab2011_2016_2021_df["FIPS str"].str[:5] + Diab2011_2016_2021_df["Year"].astype("str")
Diab2011_2016_2021_df

In [ ]:
# making merge key in the fast food dataframe

fast_food_11_16_21_df["FIPS str"] = fast_food_11_16_21_df["FIPS"].astype("str")
fast_food_11_16_21_df["merge_key"] = fast_food_11_16_21_df["FIPS str"].str[:5] + fast_food_11_16_21_df["Year"].astype("str")
fast_food_11_16_21_df

In [ ]:
# Now merge in the fast food restaurant count data into the diabetes dataframe

merge_diab_fastfood_df = pd.merge(Diab2011_2016_2021_df, fast_food_11_16_21_df, how="left", on="merge_key")
merge_diab_fastfood_df

In [ ]:
# select the columns we want

diab_fastfood_df = merge_diab_fastfood_df.loc[ :, [ "FIPS_x", "State", "County", "% Diabetic", "Population", "Year_x", "Diabetic Population","Restaurant Count"]]
diab_fastfood_df

In [ ]:
# clean up the column names

diab_fastfood_rename_df = diab_fastfood_df.rename(columns={"FIPS_x":"FIPS","Year_x":"Year"})
diab_fastfood_rename_df

In [ ]:
# some 2021 fast food counts come in blank, as counties with 0 restaurants did not get brought into the 2021 yelp data.
# this is creating nan restaurant counts for some counties in 2021.  Therefore, filling those with 0's

diab_fastfood_rename_df["Restaurant Count"] = diab_fastfood_rename_df["Restaurant Count"].fillna(0)
diab_fastfood_rename_df.head()

In [ ]:
# now removing any rows that have blanks (for example FIPS 17000 which is the entire state and not a county)

# THIS IS THE DATAFRAME TO PLOT AND ANALYZE WITH


diab_fastfood_clean_df = diab_fastfood_rename_df.dropna(how='any') 
diab_fastfood_clean_df.head()

In [ ]:
# Export the dataframe to csv
diab_fastfood_clean_df.to_csv("diab_fastfood_clean_df.csv")